In [3]:
from elasticsearch import Elasticsearch
import pickle

In [20]:
import warnings

# Ignore all warnings (not recommended in most cases)
warnings.filterwarnings("ignore")

## Connect to Elasticsearch

In [5]:
es = Elasticsearch("http://localhost:9200")
resp = es.info()
resp

ObjectApiResponse({'name': '163dd7feee2c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qiPQpuGBSwGK5RW2XA7YHw', 'version': {'number': '8.8.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98e1271edf932a480e4262a471281f1ee295ce6b', 'build_date': '2023-06-26T05:16:16.196344851Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
file_name = '.\Data\df_dict.pkl'
with open(file_name, "rb") as file:
    df_dict = pickle.load(file)

## Store the data to the elasticsearch

In [8]:
index_name = 'products'

In [9]:
# for i, product in enumerate(df_dict):
#     es.index(index=index_name, id = product['Product_id'], body=product)

## Define the count query to get the total number of documents in the index

In [10]:
count_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the count query
response = es.count(index=index_name, body=count_query)

# Extract the total count of documents
total_documents = response['count']

print(f"Total number of documents in the index '{index_name}': {total_documents}")

Total number of documents in the index 'products': 488


<ipython-input-10-55ceee5bd24a>:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.count(index=index_name, body=count_query)


## Recommend products by similarity search with keywords

In [15]:
def recommend_products(keywords, num_results=5):
    res = es.search(
        index="products",
        body={
            "query": {
                "multi_match": {
                    "query": keywords,
                    "fields": ['Product_Name', 'Product_Name_en', 'Description', 'Description_en'],
                    "type": "cross_fields",
                    "operator": "and"
                }
            },
            "size": num_results
        }
    )

    recommendations = []
    for hit in res['hits']['hits']:
        recommendations.append(hit['_source']['Product_Name'])

    return recommendations

In [26]:
recommend_products('保護')

['紙護角、護角紙、紙邊條', '紙護角', '樹根保護套', '樹苗保護罩-A', '得利淨']

## Send the DELETE request to delete the entire index

In [ ]:
# response = es.indices.delete(index=index_name)

# # Check if the operation was successful
# if response["acknowledged"]:
#     print(f"The index '{index_name}' has been deleted.")
# else:
#     print(f"Failed to delete the index '{index_name}'.")